In [1]:
import numpy as np
import codecs
import pandas as pd

In [2]:
data = pd.read_csv('train.csv')

In [3]:
data.head(1)    

,id,tid1,tid2,title1_zh,title2_zh,title1_en,title2_en,label
0,0,0,1,2017养老保险又新增两项，农村老人人人可申领，你领到了吗,警方辟谣“鸟巢大会每人领5万” 仍有老人坚持进京,There are two new old-age insurance benefits f...,"Police disprove ""bird's nest congress each per...",unrelated


In [4]:
exist = set()
mapping = {}
strange = set()

In [5]:
tid1s = data['tid1'].tolist()
tid2s = data['tid2'].tolist()
title1_zhs = data['title1_zh'].tolist()
title2_zhs = data['title2_zh'].tolist()
title1_ens = data['title1_en'].tolist()
title2_ens = data['title2_en'].tolist()
labels = data['label'].tolist()

In [6]:
def check(id1):
    for i in range(len(data_sets)):
        if(id1 in data_sets[i]):
            return i
    return -1

In [7]:
class graph:
    def __init__(self):
        self.data = {}
        self.dis_path = {}
        self.un = {}
        
    def add_link(self,a,b):
        def add(a,b):
            try:
                self.data[a].append(b)
            except:
                self.data[a] = [b]
        add(a,b)
        add(b,a)
    
    def add_dis(self,a,b):
        def dis(a,b):
            if(b not in self.path):
                self.path[b] = [b]        
            if(a in self.path[b]):
                strange.add((a,b))
                return True
            try:
                self.dis_path[a].extend(self.path[b])
            except KeyError as e:
                self.dis_path[a] = self.path[b]
            return False
        
        if(dis(a,b)):
            return 
        dis(b,a)
            
    def dfs_all(self):
        def dfs(a,check):
            ans = [a]
            check.add(a)
            for p in self.data[a]:
                if(p not in check):
                    ans.extend(dfs(p,check))
            return ans
            
            
        self.path = {}
        
        for root in self.data:
            temp = dfs(root,set())
            
            self.path[root] = temp
            
    def add_un(self,a,b):
        def un(a,b):
            
            if((b in self.path and a in self.path[b]) or (b in self.dis_path and a in self.dis_path[b])):
                strange.add((a,b))
                return True
            
            try:
                self.un[a].append(b)
            except:
                self.un[a] = [b]
            return False
        
        if(un(a,b)):
            return 
        un(b,a)
        
            

In [8]:
g = graph()
#first build up the disjoint set
count = 0
for tid1,tid2,title1_zh,title2_zh,title1_en,title2_en,label in zip(tid1s,tid2s,title1_zhs,title2_zhs,title1_ens,title2_ens,labels):
    if(tid1 not in exist):
        exist.add(tid1)
        mapping[tid1] = [title1_zhs,title1_ens]
    if(tid2 not in exist):
        exist.add(tid2)
        mapping[tid2] = [title2_zhs,title2_ens]
    
    if(tid2==42543 or tid1==42543):
        continue
    if(tid1==108313 and tid1==108312):
        continue
        
    if(label=='agreed'):
        count += 1
        g.add_link(tid1,tid2)
g.dfs_all()

In [9]:
after_count = 0
for key in g.path:
    after_count += len(g.path[key])-1
    
len(g.path),count,after_count//2

(59018, 92966, 413301)

In [10]:
count = 0
for tid1,tid2,title1_zh,title2_zh,title1_en,title2_en,label in zip(tid1s,tid2s,title1_zhs,title2_zhs,title1_ens,title2_ens,labels):
    if(label=='disagreed'):
        if(tid1 == tid2):
            continue
        count += 1
        g.add_dis(tid1,tid2)


In [11]:
after_count = 0
for key in g.dis_path:
    after_count += len(g.dis_path[key])
    
len(g.dis_path),count,after_count//2

(8414, 8256, 166692)

In [12]:
count = {'unrelated':0,'agreed':0,'disagreed':0}
for tid1,tid2,title1_zh,title2_zh,title1_en,title2_en,label in zip(tid1s,tid2s,title1_zhs,title2_zhs,title1_ens,title2_ens,labels):
    if(tid1 == tid2):
        count[label] += 1
print(count)

{'unrelated': 96, 'agreed': 4, 'disagreed': 10}


In [13]:
count = 0
for tid1,tid2,title1_zh,title2_zh,title1_en,title2_en,label in zip(tid1s,tid2s,title1_zhs,title2_zhs,title1_ens,title2_ens,labels):
    if(label=='unrelated'):
        if(tid1 == tid2):
            continue
        count += 1
        g.add_un(tid1,tid2)

In [14]:
after_count = 0
for key in g.un:
    after_count += len(g.un[key])-1
    
len(g.un),count,after_count//2

(137925, 219217, 135845)

In [15]:
data = []
for key in g.path:
    for p in g.path[key][1:]:
        if(key>p):
            data.append((key,p,'agreed'))
for key in g.dis_path:
    for p in g.dis_path[key]:
        if(key>p):
            data.append((key,p,'disagreed'))
for key in g.un:
    for p in g.un[key]:
        if(key>p):
            data.append((key,p,'unrelated'))

In [16]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2)

In [ ]:
ids = []
tid1s = []
tid2s = []
title1_zhs = []
title2_zhs = []
title1_ens = []
title2_ens = []
labels = []
for i,d in enumerate(train):
    ids.append(i)
    tid1s.append(d[0])
    tid2s.append(d[1])
    title1_zhs.append(mapping[d[0]][0])
    title2_zhs.append(mapping[d[1]][0])
    title1_ens.append(mapping[d[0]][1])
    title2_ens.append(mapping[d[1]][1])
    labels.append(d[-1])
    
pd.DataFrame({'id':ids,
           'tid1':tid1s,
           'tid2':tid2s,
           'title1_zh':title1_zhs,
           'title2_zh':title2_zhs,
           'title1_en':title1_ens,
           'title2_en':title2_ens,
           'label':labels
          }).to_csv('local_train.csv')

In [20]:
ids = []
tid1s = []
tid2s = []
title1_zhs = []
title2_zhs = []
title1_ens = []
title2_ens = []
labels = []
for i,d in enumerate(test):
    ids.append(i)
    tid1s.append(d[0])
    tid2s.append(d[1])
    title1_zhs.append(mapping[d[0]][0])
    title2_zhs.append(mapping[d[1]][0])
    title1_ens.append(mapping[d[0]][1])
    title2_ens.append(mapping[d[1]][1])
    labels.append(d[-1])
    
pd.DataFrame({'id':ids,
           'tid1':tid1s,
           'tid2':tid2s,
           'title1_zh':title1_zhs,
           'title2_zh':title2_zhs,
           'title1_en':title1_ens,
           'title2_en':title2_ens,
           'label':labels
          }).to_csv('local_test.csv')

OSError: [Errno 28] No space left on device